In [5]:
from pathlib import Path
outdir = Path.cwd().parent / "traffic simulation" / "2906"
print(outdir.exists())


True


In [6]:
# =========================
# Find entry and exit edges of a SUMO network
# =========================

import sumolib

net = sumolib.net.readNet(outdir / "osm.net.xml.gz")

entry_edges = []
exit_edges = []

for edge in net.getEdges():
    if not edge.getIncoming():   # no incoming = boundary entry
        entry_edges.append(edge.getID())
    if not edge.getOutgoing():   # no outgoing = boundary exit
        exit_edges.append(edge.getID())

print("Entry edges:", entry_edges)
print("Exit edges:", exit_edges)


Entry edges: ['-1088700963', '-7637536#1', '623950256', '624515487', '624523286']
Exit edges: ['-624523286', '1088700963', '623950255', '624515488', '7637536#0']


In [ ]:
import sys
import os
import subprocess
# Add SUMO tools to Python path
sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

import randomTrips

netfile = outdir / "osm.net.xml.gz"
tripfile = outdir / "trips.rou.xml"
SUMO_HOME = os.environ["SUMO_HOME"]

cmd = [
    "python", os.path.join(SUMO_HOME, "tools", "randomTrips.py"),
    "-n", netfile,
    "-o", tripfile,
    "-b", "0",
    "-e", "3600",
    "--period", "0.5",
    "--validate"
]

subprocess.run(cmd, check=True)
print(f"Trips written to {tripfile}")


Trips written to trips.rou.xml


In [ ]:
# =========================
# Filter trips based on forbidden (from,to) edge pairs
# =========================

from sumolib.xml import parse

# Define forbidden (from,to) pairs
forbidden_pairs = {
    ("-1088700963", "1088700963"),
    ("-7637536#1", "7637536#0"),
    ("623950256", "623950255"),
    ("624515487", "624515488"),
    ("624523286", "-624523286"),
}

#Define acceptable edges
acceptable_edges = {'-1088700963', '-7637536#1', '623950256', '624515487', '624523286','-624523286', '1088700963', '623950255', '624515488', '7637536#0'}

input_file  = outdir / "trips.rou.xml"
output_file = outdir / "filtered_trips.rou.xml"

with open(output_file, "w") as out:
    out.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" '
              'xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')

    for trip in parse(input_file, "trip"):
        start = getattr(trip, "attr_from")  # 'from' edge
        end   = getattr(trip, "to")    # 'to' edge
        if start in acceptable_edges and end in acceptable_edges:
            if (start, end) not in forbidden_pairs:
                out.write(trip.toXML("\t"))
                # out.write("\n")

    out.write("</routes>\n")

print(f"Filtered trips written to {output_file}")


Filtered trips written to filtered_trips.rou.xml


In [ ]:
# =========================
# find all edges 
# =========================

from sumolib.xml import parse
temp = outdir / "filtered_trips.rou.xml"

edges = []

for trip in parse(temp, "trip"):
    start = getattr(trip, "attr_from")  # 'from' edge
    end   = getattr(trip, "to")    # 'to' edge
    edges.append((start, end))
    # print(start, end)

# =========================
# find unique edges
# =========================
unique_edges = set()
for edge_pair in edges:
    unique_edges.update(edge_pair)  
print("Unique edges:", unique_edges)
print("Number of unique edges:", len(unique_edges))


Unique edges: {'624515488', '7637536#0', '624523286', '624515487', '623950255', '-7637536#1', '623950256', '-624523286', '-1088700963', '1088700963'}
Number of unique edges: 10


In [8]:
import random

def generate_linear_trips(output_file, entries, exits, forbidden_pairs,
                          num_trips=50, begin=0, period=2):
    """
    Generate exactly num_trips trips with linear depart times.
    Constraints:
      - from ∈ entries
      - to ∈ exits
      - (from,to) not in forbidden_pairs
    """
    entries = list(entries)
    exits = list(exits)

    with open(output_file, "w") as out:
        out.write('<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" '
                  'xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n')

        trip_id = 0
        i = 0
        while trip_id < num_trips:
            depart = round(begin + i * period, 2)
            start = random.choice(entries)
            end_edge = random.choice(exits)

            # retry until valid combination
            while (start, end_edge) in forbidden_pairs:
                start = random.choice(entries)
                end_edge = random.choice(exits)

            out.write(f'    <trip id="{trip_id}" depart="{depart}" from="{start}" to="{end_edge}"/>\n')
            trip_id += 1
            i += 1

        out.write("</routes>\n")

    print(f"✅ Generated {trip_id} trips in {output_file}")


In [9]:
entry_edges = ['-1088700963', '-7637536#1', '623950256', '624515487', '624523286']
exit_edges  = ['-624523286', '1088700963', '623950255', '624515488', '7637536#0']

forbidden_pairs = {
    ("-1088700963", "1088700963"),
    ("-7637536#1", "7637536#0"),
    ("623950256", "623950255"),
    ("624515487", "624515488"),
    ("624523286", "-624523286"),
}

number_of_trips = 2000
frequent_trips = 1

generate_linear_trips(
    output_file= outdir / f"trips-{number_of_trips}-{frequent_trips}.trips.xml", ## change name here
    entries=entry_edges,
    exits=exit_edges,
    forbidden_pairs=forbidden_pairs,
    num_trips=number_of_trips,
    begin=0,
    period=frequent_trips # frequent trips 
)


✅ Generated 2000 trips in c:\Users\hanju\OneDrive - AUT University\01. Papers\09. Dissertation\4.0 Projects\xai-traffic-prediction\traffic simulation\2906\trips-2000-1.trips.xml


In [ ]:
import os, subprocess

SUMO_HOME = os.environ["SUMO_HOME"]
netfile = outdir / "osm.net.xml.gz"
outfile = outdir / "detectors.add.xml"

cmd = [
    "python",
    os.path.join(SUMO_HOME, "tools", "output", "generateTLSE1Detectors.py"),
    "-n", netfile,
    "-o", outfile
]

subprocess.run(cmd, check=True)
print(f"✅ Detectors written to {outfile}")


✅ Detectors written to detectors.add.xml


In [ ]:
import sumolib

net = sumolib.net.readNet("osm.net.xml.gz")

with open(outdir /"detectors.add.xml", "w") as f:
    f.write("<additional>\n")
    for edge in net.getEdges():
        for lane in edge.getLanes():
            det_id = f"loop_{lane.getID()}"
            f.write(f'    <inductionLoop id="{det_id}" lane="{lane.getID()}" pos="50" freq="60" file="detector_output.xml"/>\n')
    f.write("</additional>\n")

print("✅ All-lane detectors written to detectors.add.xml")


✅ All-lane detectors written to detectors.add.xml
